In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import matplotlib.pyplot as plt

def dealImg(img):
    b, g, r = cv2.split(img)
    img_rgb = cv2.merge([r, g, b])
    return img_rgb

path = './'  # 原图路径
# cropped_save_path = 'cropped_pic/' + img_path  # 眼部区域存储路径


model_path = 'face_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)
landmarker = FaceLandmarker.create_from_options(options)

imgs_ = os.listdir(path)

imgs = []
for i in imgs_:
    if '2' in i:
        imgs.append(i)
        
# with open('list.txt', 'w') as file:
#     file.write(';'.join(str(element) for element in imgs))

plt.figure(figsize=(5184/96, 3888/96), dpi=96)

for img_list in imgs:
    cropped_save_path = 'eyes_blur/' + img_list
    imgs_list = os.listdir(img_list)
    for img in imgs_list:
        mp_image = mp.Image.create_from_file(img_list + '/' + img)
        face_landmarker_result = landmarker.detect(mp_image)
        mp_width = mp_image.width
        mp_height = mp_image.height
        image = cv2.imread(img_list + '/' + img)
        if not face_landmarker_result.face_landmarks:
            print(img_list + '/' + img)
        else:
            # 获取得到的特征点坐标
            landmarks = face_landmarker_result.face_landmarks[0]
            left_up_x = int(landmarks[53].x * mp_width)
            left_up_y = int(landmarks[53].y * mp_height)
            right_down_x = int(landmarks[346].x * mp_width)
            right_down_y = int(landmarks[346].y * mp_height)
            eye_region_width = right_down_x - left_up_x
            eye_region_height = int(0.375 * eye_region_width)
            fix_ycenter = int((right_down_y + left_up_y) * 0.5)
            fix_left_up_y = fix_ycenter - int(0.5 * eye_region_height)
            
            # 颜色变换
            mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)
            left_eye_point = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]
            right_eye_point = [263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249]
            left_eye_mask = []
            right_eye_mask = []
            for i in left_eye_point:
                left_eye_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
            for i in right_eye_point:
                right_eye_mask.append([int(landmarks[i].x * mp_width), int(landmarks[i].y * mp_height)])
                
            lefteye_mask_temp = cv2.fillConvexPoly(mask, np.int32(left_eye_mask), (1.0, 1.0, 1.0))
            righteye_mask_temp = cv2.fillConvexPoly(mask, np.int32(right_eye_mask), (1.0, 1.0, 1.0))
            mask += lefteye_mask_temp
            mask += righteye_mask_temp
        
            mask = 255 * np.uint8(mask)
            kernel = cv2.getStructuringElement(cv2.MORPH＿RECT, (40, 40))
            mask = cv2.morphologyEx(mask, cv2.MORPH＿CLOSE, kernel, 1)
            mask = cv2.GaussianBlur(mask, (15, 15), cv2.BORDER＿DEFAULT)
            # 对图像进行按位取反操作
            inverseMask = cv2.bitwise＿not(mask)
            # 将掩码转换为浮点以执行混合
            mask = mask.astype(float)/255 
            inverseMask = inverseMask.astype(float)/255
            # 颜色映射
            lut = np.zeros((256,1,3), dtype=np.uint8)  # 建立查找表
            for i in range(256):
                if i == 0:
                    lut[i] = (0,0,0)
                else:
                    tmp = int(np.exp(np.log(i / 255.0) * 0.7) * 255.0)  # dst = K*E^r = 255 * E^(log(p(x,y)/255)*gamma
                    lut[i]=(tmp,tmp,tmp)
            eyes = cv2.LUT(image,lut)
            # 将eyes和人脸转换为0-1范围
            eyes = eyes.astype(float)/255 
            ladyFace = image.astype(float)/255
            # 用掩码调整eyes和脸
            justeyes = cv2.multiply(mask, eyes)
            justFace = cv2.multiply(inverseMask, ladyFace)
            # 加上脸和eyes
            result = justFace + justeyes
        
            
                # plt.figure(figsize=(5184/96, 3888/96), dpi=96)
            plt.imshow(dealImg(result), 'gray')
            plt.axis('off')
            plt.savefig(os.path.join(cropped_save_path, img), dpi=96, bbox_inches='tight', pad_inches = 0)
            plt.clf()